In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import matplotlib
from tqdm  import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36'
}
GEO = 'united-states'

In [2]:
driver = webdriver.Chrome()
driver.get(f'https://tradingeconomics.com/{GEO}/indicators')

In [3]:
macro = driver.find_elements(By.XPATH, '//*[@id="pagemenutabs"]')[0].text.split('\n')[:-1]

In [4]:
rows = []
for l in range(0, len(macro)):
    driver.find_element(By.XPATH, f'//*[@id="pagemenutabs"]/li[{l+1}]/a').click()
    data = driver.find_elements(By.XPATH, f'//*[@id="{macro[l].lower()}"]/div/div/table/tbody')[0].text.replace('30 Year Mortgage Rate', 'thirty Year Mortgage Rate').replace('15 Year Mortgage Rate', 'fifteen Year Mortgage Rate').replace('Michigan 5 Year Inflation Expectations', 'Michigan five Year Inflation Expectations').replace('1000', 'one thousand').replace('4-week','four-week').split('\n')
    
    for i in range(0,len(data)):
        tmp = data[i].split(' ')
        name = ''
        unit = ''
        notice_date = ''
        last = ''
        previous = ''
        highest = ''
        lowest = ''
        signal = 0
        pre = 0
        for j in range(0,len(tmp)):
            # j = pre
            if signal == 0:
                if ord(tmp[j][0])>=65: # name
                    name += tmp[j]+' '
                else: # 數字們
                    signal = 1
                    k = j

            if signal == 1 and ord(tmp[j][0])>=65:
                if '/' not in tmp[j]:
                    unit += tmp[j]+' '
                else:
                    signal = 2
                    pre = j
            if signal == 2:
                if '/' in tmp[j]:
                    notice_date = tmp[j]
            
            
        name = name[:-1]
        unit = unit[:-1]
        notice_date = notice_date[:-1]

        try:
            if '/' not in tmp[k+1] and ord(tmp[k+1][0])<65:
                previous += tmp[k+1]
            else:
                previous = ''
        except:
            previous = ''
        
        try:
            if '/' not in tmp[k+2] and ord(tmp[k+2][0])<65:
                highest += tmp[k+2]
        except:
            highest = ''
          
        try:
            if '/' not in tmp[k+3]and ord(tmp[k+3][0])<65:
                lowest += tmp[k+3]
        except:
            lowest = ''
        
            
        rows.append({'總經種類':macro[l],'指標名稱':name, 'Last':tmp[k], 'Previous': previous, 'Higest': highest, 'Lowest': lowest, '單位': unit, '公告日期': notice_date})

In [5]:
data = pd.DataFrame(rows)

In [8]:
data.to_excel(f'5_經濟數據_{GEO}.xlsx', index = 0)